# Introduction to the Python control system

This example covers the basic introduction to the control of [Zahner ZENNIUM series potentiostats](https://zahner.de/products#potentiostats) with [Python](https://www.python.org/). It explains in detail how to connect to the Term Terminal Software running the Thales Workstation Software and how to perform measurements.

Zahner does not offer an introduction to Python programming, but Python has a [BeginnersGuide website](https://wiki.python.org/moin/BeginnersGuide) that will help you get started quickly.

There is help available for different entry levels, for [non-programmers](https://wiki.python.org/moin/BeginnersGuide/NonProgrammers) or for those with [programming experience](https://wiki.python.org/moin/BeginnersGuide/Programmers).
Here are several tutorials for the entry level listed, such as [learnpython.org](https://www.learnpython.org/). The Python [BeginnersGuide](https://wiki.python.org/moin/BeginnersGuide) also lists different integrated development environments that can be used to edit, develop and debug the Python code. Here we can recommend [Visual Studio Code](https://code.visualstudio.com/) with the [Python extension](https://marketplace.visualstudio.com/items?itemName=ms-python.python) or Eclipse with the [PyDev extension](https://www.pydev.org/).

Besides the examples in the [GitHub repository](https://github.com/Zahner-elektrik/Thales-Remote-Python) there is a [documentation page](https://doc.zahner.de/thales_remote/) where all functions of the library are explained.

In [1]:
import sys
import math
import cmath
from thales_remote.connection import ThalesRemoteConnection
from thales_remote.script_wrapper import PotentiostatMode, ThalesRemoteScriptWrapper

# Utility functions

First, two utility functions are declared.  
With the first function the complex impedance is output to the console. The second function measures an impedance spectrum from individual impedance measuring points.

In [2]:
def printImpedance(impedance):
    print(
        f"Impedance: {abs(impedance):>10.3e} ohm {cmath.phase(impedance)/cmath.pi*180.0:>10.2f} degree"
    )
    return


def spectrum(scriptHandle, lower_frequency, upper_frequency, number_of_points):
    log_lower_frequency = math.log(lower_frequency)
    log_upper_frequency = math.log(upper_frequency)
    log_interval_spacing = (log_upper_frequency - log_lower_frequency) / (
        number_of_points - 1
    )

    for i in range(number_of_points):
        current_frequency = math.exp(log_lower_frequency + log_interval_spacing * i)
        print(f"Frequency: {current_frequency:e} Hz")
        printImpedance(scriptHandle.getImpedance(current_frequency))

    return

# Connect to the Term software

Before calling the following commands, the Term software must be started and the Thales start screen must be displayed, in which methods can be selected.

In this case the Term runs on the same computer as Python, so **"localhost"** is chosen here as IP address, but you could also specify the IP address of the computer in the network, on which the Term software is running and to which the workstation is connected via USB.

In [3]:
TARGET_HOST = "localhost"

zenniumConnection = ThalesRemoteConnection()
zenniumConnection.connectToTerm(TARGET_HOST)

True

# Initialize the application and start the remote software in Thales

If the connection to the Term is successfully established, the [ThalesRemoteConnection](https://doc.zahner.de/thales_remote/connection.html#thales_remote.connection.ThalesRemoteConnection) object manages the connection to the Term software. This object is passed to the constructor of a new [ThalesRemoteScriptWrapper](https://doc.zahner.de/thales_remote/script_wrapper.html#thales_remote.script_wrapper.ThalesRemoteScriptWrapper) object.

The [ThalesRemoteScriptWrapper](https://doc.zahner.de/thales_remote/script_wrapper.html#thales_remote.script_wrapper.ThalesRemoteScriptWrapper) class contains the commands of the [Remote2](https://doc.zahner.de/manuals/remote2.pdf) as [getter and setter methods](https://en.wikipedia.org/wiki/Mutator_method).

In [ ]:
zahnerZennium = ThalesRemoteScriptWrapper(zenniumConnection)
zahnerZennium.forceThalesIntoRemoteScript()

# Offset calibration

The first step is to perform an offset calibration, this takes only a few seconds.  
The instrument performs automatic offset calibrations but it is recommended to calibrate the offsets manually after the instrument has warmed up for half an hour.

In [5]:
zahnerZennium.calibrateOffsets()

'OK\r'

# DC measurement

## Potentiostatic measurement

A voltage of 1.0 V is output potentiostatically as an example. For this, potentiostatic mode is set first. Then the potential is set and the potentiostat is switched on.

In [6]:
zahnerZennium.setPotentiostatMode(PotentiostatMode.POTMODE_POTENTIOSTATIC)
zahnerZennium.setPotential(1.0)
zahnerZennium.enablePotentiostat()

'OK\r'

After the potentiostat is switched on, voltage and current are measured several times in a loop.

Here the voltage and the current are read with the getters mentioned before.

In [7]:
for i in range(5):
    print(f"Potential:\t{zahnerZennium.getPotential():>10.6f} V")
    print(f"Current:\t{zahnerZennium.getCurrent():>10.3e} A")

Potential:	  0.999915 V
Current:	 1.003e-08 A
Potential:	  0.999904 V
Current:	 1.005e-08 A
Potential:	  0.999902 V
Current:	 1.005e-08 A
Potential:	  0.999900 V
Current:	 1.005e-08 A
Potential:	  0.999901 V
Current:	 1.005e-08 A


## Galvanostatic measurement

After the potentiostatic measurement, galvanostatic measurement is performed with 20 nA.

In [8]:
zahnerZennium.disablePotentiostat()
zahnerZennium.setPotentiostatMode(PotentiostatMode.POTMODE_GALVANOSTATIC)
zahnerZennium.setCurrent(20e-9)
zahnerZennium.enablePotentiostat()

for i in range(5):
    print(f"Potential:\t{zahnerZennium.getPotential():>10.6f} V")
    print(f"Current:\t{zahnerZennium.getCurrent():>10.3e} A")

Potential:	  1.982775 V
Current:	 2.004e-08 A
Potential:	  1.995354 V
Current:	 2.004e-08 A
Potential:	  1.995975 V
Current:	 2.004e-08 A
Potential:	  1.996287 V
Current:	 2.004e-08 A
Potential:	  1.996538 V
Current:	 2.004e-08 A


# Impedance measurement

## Parameterization of the impedance measurement

For the impedance measurement, the measuring frequency, the measuring amplitude and the number of periods to be averaged are now set.

As explained in the [Remote2 manual on page 15](https://doc.zahner.de/manuals/remote2.pdf#page=15), after switching on the potentiostat, the current must be measured with [getCurrent()](https://doc.zahner.de/thales_remote/script_wrapper.html?highlight=getcurrent#thales_remote.script_wrapper.ThalesRemoteScriptWrapper.getCurrent), which sets the optimum current range for the impedance measurement.

Likewise, an amplitude must be set for the impedance measurement. The amplitude must be switched off manually when it is not required.

In [9]:
zahnerZennium.disablePotentiostat()
zahnerZennium.setPotentiostatMode(PotentiostatMode.POTMODE_POTENTIOSTATIC)
zahnerZennium.setPotential(1.0)
zahnerZennium.enablePotentiostat()
zahnerZennium.setFrequency(2000)
zahnerZennium.setNumberOfPeriods(3)

zahnerZennium.enablePotentiostat()
zahnerZennium.getCurrent()

zahnerZennium.setAmplitude(10e-3)

'OK\r'

## Execute the measurement

Since the potentiostat is still switched on from the DC measurement, the impedance spectrum is now measured at the set DC potential of 1 V.

<div class="alert alert-block alert-info">
<b>Note:</b> Only with impedance spectra is it possible to start a potentiostatic measurement automatically on the OCP by starting the measurement with the potentiostat switched off. For single impedance measurement points, the voltage must be measured and then set as the potentiostatic value.
</div>
    

In [10]:
printImpedance(zahnerZennium.getImpedance())
printImpedance(zahnerZennium.getImpedance(2000))
printImpedance(zahnerZennium.getImpedance(2000, 10e-3, 3))

spectrum(zahnerZennium, 1000, 2e5, 10)

Impedance:  5.939e+05 ohm     -89.57 degree
Impedance:  6.019e+05 ohm     -89.53 degree
Impedance:  5.949e+05 ohm     -89.10 degree
Frequency: 1.000000e+03 Hz
Impedance:  1.188e+06 ohm     -89.36 degree
Frequency: 1.801648e+03 Hz
Impedance:  6.602e+05 ohm     -90.16 degree
Frequency: 3.245936e+03 Hz
Impedance:  3.636e+05 ohm     -90.79 degree
Frequency: 5.848035e+03 Hz
Impedance:  1.993e+05 ohm     -90.97 degree
Frequency: 1.053610e+04 Hz
Impedance:  1.096e+05 ohm     -90.75 degree
Frequency: 1.898235e+04 Hz
Impedance:  6.064e+04 ohm     -90.39 degree
Frequency: 3.419952e+04 Hz
Impedance:  3.364e+04 ohm     -90.22 degree
Frequency: 6.161550e+04 Hz
Impedance:  1.865e+04 ohm     -90.11 degree
Frequency: 1.110095e+05 Hz
Impedance:  1.035e+04 ohm     -89.99 degree
Frequency: 2.000000e+05 Hz
Impedance:  5.753e+03 ohm     -90.01 degree


## Switch off potentiostat

After the measurement, the potentiostat is switched off and **the amplitude must be set to 0** again after the impedance measurement.

In [11]:
zahnerZennium.disablePotentiostat()
zahnerZennium.setAmplitude(0)

'OK\r'

# Close the connection

In [12]:
zenniumConnection.disconnectFromTerm()
print("finish")

finish
